In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_113783/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_3_9_15,0.998739,0.995000,0.992071,0.994109,0.152117,0.427947,0.539341,0.480367,"Hidden Size=[25], regularizer=0.01, learning_r..."
1,model_3_9_14,0.998739,0.995028,0.992183,0.994169,0.152126,0.425599,0.531708,0.475532,"Hidden Size=[25], regularizer=0.01, learning_r..."
2,model_3_9_16,0.998737,0.994975,0.991968,0.994055,0.152272,0.430128,0.546343,0.484817,"Hidden Size=[25], regularizer=0.01, learning_r..."
3,model_3_9_13,0.998737,0.995057,0.992305,0.994233,0.152359,0.423083,0.523411,0.470296,"Hidden Size=[25], regularizer=0.01, learning_r..."
4,model_3_9_17,0.998735,0.994951,0.991874,0.994005,0.152545,0.432151,0.552754,0.488906,"Hidden Size=[25], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
1180,model_14_3_5,0.971612,0.968002,0.973661,0.978276,3.423626,4.428453,1.494776,3.047901,"Hidden Size=[10, 5], regularizer=0.1, learning..."
1191,model_13_8_4,0.971298,0.959196,0.979987,0.967435,3.461442,6.479716,0.648224,3.735483,"Hidden Size=[10, 5], regularizer=0.01, learnin..."
1337,model_13_8_3,0.967863,0.963879,0.986015,0.971668,3.875770,5.736067,0.452974,3.249902,"Hidden Size=[10, 5], regularizer=0.01, learnin..."
1341,model_14_3_4,0.967748,0.969940,0.980329,0.980558,3.889609,4.160190,1.116363,2.727801,"Hidden Size=[10, 5], regularizer=0.1, learning..."
